# 02 Chat Completion | 02 .NET SDK

## Azure Environment

To execute the sample code Azure service specific information like endpoint, api key etc. is needed ([Details and instructions can be found here](../01_CreateEnvironment/01_Environment.ipynb))

## Step 1: Create OpenAIClient

The OpenAIClient from Azure.AI.OpenAI is a .NET client library that acts as the centralized point for all .NET functionality that want to interact with a deployed Azure OpenAI Large Language Model. It provides methods to access the OpenAI REST APIs for various tasks such as text completion, text embedding, and chat completion, etc.. It also allows developers to specify the model, engine, and options for each request, such as temperature, frequency penalty, presence penalty, and stop sequences. 

The OpenAIClient can connect to any Azure OpenAI resource or to the non-Azure OpenAI inference endpoint, making it a versatile and powerful tool for .NET development with OpenAI.

In [ ]:
#r "nuget: Azure.AI.OpenAI, 2.0.0-beta.2"
#r "nuget: DotNetEnv, 2.5.0"

using Azure; 
using Azure.AI.OpenAI;
using OpenAI.Chat;
using DotNetEnv;
using System.IO;
using System.Text.Json; 

//configuration file is created during environment creation
static string _configurationFile = @"../Configuration/application.env";
Env.Load(_configurationFile);


string assetsFolder = Path.Combine(Directory.GetCurrentDirectory(), "..", "..", "assets");

string oAiApiKey = Environment.GetEnvironmentVariable("WS_AOAI_APIKEY") ?? "WS_AOAI_APIKEY not found";
string oAiEndpoint = Environment.GetEnvironmentVariable("WS_AOAI_ENDPOINT") ?? "WS_AOAI_ENDPOINT not found";
string chatCompletionDeploymentName = Environment.GetEnvironmentVariable("WS_CHATCOMPLETION_DEPLOYMENTNAME") ?? "WS_CHATCOMPLETION_DEPLOYMENTNAME not found";

AzureKeyCredential azureKeyCredential = new AzureKeyCredential(oAiApiKey);
AzureOpenAIClient azureOpenAIClient = new AzureOpenAIClient(new Uri(oAiEndpoint), azureKeyCredential);
ChatClient chatClient = azureOpenAIClient.GetChatClient(chatCompletionDeploymentName);

Console.WriteLine($"AzureOpenAI Client created...");
Console.WriteLine($"ChatClient created...");

**Expected output:**

Installed Packages

  - Azure.AI.OpenAI, 2.0.0-beta.2
  - DotNetEnv, 2.5.0

AzureOpenAI client created ...

ChatClient created ...

## Step 1: Basic Interaction

The following cell, demonstrate a basic interaction with OpenAI Client. In this case, the system message is used to provide instructions or settings for the assistant, such as its personality or behavior. The user message is used to provide queries or inputs from the human user. The assistant message is used to provide responses or outputs from the model.

System prompt: `"You are an AI assistance who helps finding information around international sport events"`

Prompt: `"Who won the super bowl 2024?"`

In [ ]:
string systemMessage = "You are an AI assistance who helps finding information around international sport events";
string userMessage = "Who won the super bowl 2024?";

ChatCompletionOptions chatComletionOptions = new ChatCompletionOptions(){
    MaxTokens = 500,
    Temperature = 0.0f,
    TopP = 1.0f,
    FrequencyPenalty = 0.7f,
    PresencePenalty = 0.7f,
};
chatComletionOptions.StopSequences.Add("\n");

ChatCompletion chatCompletion = await chatClient.CompleteChatAsync(
    messages: [
        new SystemChatMessage(systemMessage),
        new UserChatMessage(userMessage)
    ], 
    options: chatComletionOptions
);

Console.WriteLine($"Model response: \n {chatCompletion.Content[0]}"); 


**Expected outcome:**

Model response:

[Some information that the event will happen in the future]

## Step 2: Complex interaction - Chat History

### Prepare chat history

System Prompt: 
```html
    You extract intention from provided text. The two intentions you identify are: product information and order status. 
    You answer with a valid JSON string. 
    The JSON string must have the format {""Intention"": ""ProductInformation""} or {""Intention"": ""OrderInfo""}. 
    You don't provide additional information. 
    If you can't identify intention answer with {""Intention"": ""Unknown""}
```

Chat History: 

```html
    - I've purchased three weeks ago new training shoes. When will they will be delivered?
    - Still waiting for the delivery. Any idea when it will arrive? I'm Robert and I'm calling on behalf of company Contoso.
    - Do you have training shoes? If yes, I'm interested in running equipment specifically running shoes.
    - What is the average price for running shoes?
```



In [ ]:
// Define System Prompt
string systemMessage = @" 
    You extract intention from provided text. The two intentions you identify are: product information and order status. 
    You answer with a valid JSON string. 
    The JSON string must have the format {""Intention"": ""ProductInformation""} or {""Intention"": ""OrderInfo""}. 
    You don't provide additional information. 
    If you can't identify intention answer with {""Intention"": ""Unknown""}
";

List<ChatMessage> chatMessages = new List<ChatMessage>();
chatMessages.Add(new SystemChatMessage(systemMessage));
chatMessages.Add(new UserChatMessage("I've purchased three weeks ago new training shoes. When will they will be delivered?"));
chatMessages.Add(new UserChatMessage("Still waiting for the delivery. Any idea when it will arrive? I'm Robert and I'm calling on behalf of company Contoso."));
chatMessages.Add(new UserChatMessage("Do you have training shoes? If yes, I'm interested in running equipment specifically running shoes."));
chatMessages.Add(new UserChatMessage("What is the average price for running shoes?"));

Console.WriteLine($"Chat messages created ...");


**Expected outcome:**

Chat messages created ...

### Model call

The complex chat history is provided to the model for completion

In [ ]:
ChatCompletionOptions chatCompletionOptions = new ChatCompletionOptions(){
    MaxTokens = 500,
    Temperature = 0.0f,
    TopP = 1.0f,
    FrequencyPenalty = 0.7f,
    PresencePenalty = 0.7f,
};
chatCompletionOptions.StopSequences.Add("\n");

ChatCompletion chatCompletion = await chatClient.CompleteChatAsync(
    messages: chatMessages, 
    options: chatComletionOptions
);

foreach (ChatMessageContentPart chatMessageContentPart in chatCompletion.Content){
    Console.WriteLine($"Model response: \n {chatMessageContentPart}"); 
}

**Expected outcome:**

Model response: 

{"Intention": ProductInformation}

## Step 3: Check additional response values

### Token Usage

Provide tokens consumed by the `GetChatCompletionsAsync()` call.

In [ ]:
Console.WriteLine($"Input Tokens: {chatCompletion.Usage.InputTokens}");
Console.WriteLine($"Output Tokens: {chatCompletion.Usage.OutputTokens}");
Console.WriteLine($"Total Tokens:  {chatCompletion.Usage.TotalTokens}");

**Expected outcome:**

Input Tokens: 174

Output Tokens: 8

Total Tokens: 182

### Filter 

If the LLM detects:
- hate
- indirect attach
- jailbreak
- profanity
- self harm
- secual
- violence
  
content within the provided prompt this is indicated in the response.

If the LLM detects:
- hate
- profanity
- protected code
- protected text
- self harm
- sexual
   
content within the provided results this is indicated as well.

**Attention:** Ín the current SDK this is Alpha functionality - Expect changes in the final version


In [ ]:
//Alpha Feature - Suppress diagnostics message
#pragma warning disable AOAI001
ContentFilterResultForPrompt contentFilterResultForPrompt = AzureChatCompletionExtensions.GetContentFilterResultForPrompt(chatCompletion);

Console.WriteLine($"Prompt; Hate: filtered/severity: {contentFilterResultForPrompt?.Hate?.Filtered} - {contentFilterResultForPrompt?.Hate?.Severity}");
Console.WriteLine($"Prompt; IndirectAttack: {contentFilterResultForPrompt?.IndirectAttack?.Detected} - {contentFilterResultForPrompt?.IndirectAttack?.Filtered}");
Console.WriteLine($"Prompt; Jailbreak: {contentFilterResultForPrompt?.Jailbreak?.Detected} - {contentFilterResultForPrompt?.Jailbreak?.Filtered}");
Console.WriteLine($"Prompt; Profanity: {contentFilterResultForPrompt?.Profanity?.Detected} - {contentFilterResultForPrompt?.Profanity?.Filtered}");
Console.WriteLine($"Prompt; SelfHarm: {contentFilterResultForPrompt?.SelfHarm?.Filtered} - {contentFilterResultForPrompt?.SelfHarm?.Severity}");
Console.WriteLine($"Prompt; Sexual: {contentFilterResultForPrompt?.Sexual?.Filtered} - {contentFilterResultForPrompt?.Sexual?.Severity}");
Console.WriteLine($"Prompt; Violence: {contentFilterResultForPrompt?.Violence?.Filtered} - {contentFilterResultForPrompt?.Violence?.Severity}");

ContentFilterResultForResponse contentFilterResultForResponse = AzureChatCompletionExtensions.GetContentFilterResultForResponse(chatCompletion);
Console.WriteLine($"Response; Hate: filtered/severity: {contentFilterResultForResponse?.Hate?.Filtered} - {contentFilterResultForResponse?.Hate?.Severity}");
Console.WriteLine($"Response; Profanity: {contentFilterResultForResponse?.Profanity?.Detected} - {contentFilterResultForResponse?.Profanity?.Filtered}");
Console.WriteLine($"Response; ProtectedMaterialCode: {contentFilterResultForResponse?.ProtectedMaterialCode?.Detected} - {contentFilterResultForResponse?.ProtectedMaterialCode?.Filtered}");
Console.WriteLine($"Response; ProtectedMaterialText: {contentFilterResultForResponse?.ProtectedMaterialText?.Detected} - {contentFilterResultForResponse?.ProtectedMaterialText?.Filtered}");
Console.WriteLine($"Response; SelfHarm: {contentFilterResultForResponse?.SelfHarm?.Filtered} - {contentFilterResultForResponse?.SelfHarm?.Severity}");
Console.WriteLine($"Response; Sexual: {contentFilterResultForResponse?.Sexual?.Filtered} - {contentFilterResultForResponse?.Sexual?.Severity}");
#pragma warning restore AOAI001

**Expected result:**

Prompt; Hate: filtered/severity: False - safe

Prompt; IndirectAttack:  - 

Prompt; Jailbreak:  - 

Prompt; Profanity:  - 

Prompt; SelfHarm: False - safe

Prompt; Sexual: False - safe

Prompt; Violence: False - safe

Response; Hate: filtered/severity: False - safe

Response; Profanity:  - 

Response; ProtectedMaterialCode:  - 

Response; ProtectedMaterialText:  - 

Response; SelfHarm: False - safe

Response; Sexual: False - safe
